In [3]:
!pip install unstructured[pdf]
# NOTE: Additionally, you will need to modify the app's Dockerfile to install two libraries
# Add these lines after `&& apt-get install --no-install-recommends --yes \`
#
#    libgl1-mesa-glx \ 
#    libglib2.0-0 \
#
# then run `make init`

import json
from IPython.display import display, Markdown
from unstructured.partition.pdf import partition_pdf
import requests
import tempfile

# Build a tree from the JSON returned by Unstructured
def get_tree_of_elements(unstructured_json):
    elements_by_id = {item['element_id']: item for item in unstructured_json}

    # Add children_ids
    for element_id in elements_by_id:
        if parent_id := elements_by_id[element_id]['metadata'].get('parent_id'):
            if 'children_ids' in elements_by_id[parent_id]:
                elements_by_id[parent_id]['children_ids'].append(element_id)
            else:
                elements_by_id[parent_id]['children_ids'] = [element_id]
    
    def build_tree(elements_by_id, element_ids):
        tree = []
    
        for element_id in element_ids:
            element = elements_by_id[element_id]
            if children_ids := element.get('children_ids'):
                element['children'] = build_tree(elements_by_id, children_ids)
            else:
                element['children'] = []
    
            tree.append(element)
    
        return tree

    root_element_ids = [element_id for element_id in elements_by_id if elements_by_id[element_id]['metadata'].get('parent_id') == None]
    return build_tree(elements_by_id, root_element_ids)

# Return a Markdown nested list of items in the tree
def get_tree_as_markdown(tree, indent=0):
    txt = ""
    for item in tree:
        txt += ("    " * indent) + " - [" + item['type'] + "] " + item['text'][:50] + "\n"
        txt += get_tree_as_markdown(item['children'], indent+1)
    return txt

def display_tree(tree):
    display(Markdown(get_tree_as_markdown(tree)))

def get_json_from_file(filepath):
    elements = partition_pdf(filepath, strategy="fast")
    return [element.to_dict() for element in elements]

def get_file_from_url(url):
    response = requests.get(url)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        temp_file.write(response.content)
    return temp_file.name

def get_json_from_url(url):
    temp_file_path = get_file_from_url(url)
    return get_json_from_file(temp_file_path)


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
sample_pdf = "../documents/SamplePDF.pdf"
bem_pdf = get_file_from_url("https://dhhs.michigan.gov/olmweb/ex/BP/Public/BEM/100.pdf")

pdf_json = get_json_from_file(bem_pdf)
#pdf_json = get_json_from_file(sample_pdf)

json_tree = get_tree_of_elements(pdf_json)
display_tree(json_tree)


 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 1 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [Title] OVERVIEW
     - [Title] Family Independence Program (FIP)
         - [NarrativeText] Temporary Assistance to Needy Families (TANF), cal
         - [NarrativeText] Each state must submit a state plan for FIP. State
     - [Title] Program Goal
         - [NarrativeText] The Family Independence Program (FIP) provides fin
         - [NarrativeText] Several nonfinancial and financial eligibility fac
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 2 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [Title] Medical Assistance Program
         - [NarrativeText] The Medical Assistance Program was established by 
         - [NarrativeText] HHS develops and issues federal regulations that s
     - [Title] Program Goal
         - [NarrativeText] MA provides medical assistance to individuals and 
         - [NarrativeText] Disability related Medicaid and Group 2 Medicaid e
     - [Title] Food Assistance Program (FAP)
         - [NarrativeText] The Food Assistance Program (FAP) was established 
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 3 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [NarrativeText] level. Within USDA, the Food and Nutrition Service
     - [Title] Program Goal
         - [NarrativeText] The purpose of the Food Assistance Program (FAP) i
         - [NarrativeText] FAP benefits are not considered income or assets f
     - [Title] Authorized Purchases
         - [NarrativeText] Food assistance benefits can be used to buy eligib
         - [NarrativeText] Eligible food includes:
         - [ListItem] Any food or food product intended for human consum
         - [ListItem] Seeds and plants to grow food for personal consump
         - [ListItem] Meals prepared by organizations approved by FNS as
         - [ListItem] Meals prepared and served to eligible residents by
         - [NarrativeText] Retail food stores include:
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 4 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [ListItem] Recognized grocery stores.
     - [ListItem] House-to-house grocery vendors, such as milk and m
     - [ListItem] Nonprofit food purchasing ventures - private nonpr
     - [NarrativeText] Approved meal providers may include:
     - [ListItem] Communal dining facilities for elderly and disable
     - [ListItem] Restaurants which provide low-cost meals.
     - [ListItem] Meal delivery services - public or private nonprof
     - [Title] REFUGEE ASSISTANCE PROGRAMS
         - [NarrativeText] The refugee assistance programs were established b
     - [Title] Program Goal
         - [NarrativeText] The refugee assistance programs provide financial 
         - [NarrativeText] The Immigration and Nationality Act, the Code of F
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 5 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [Title] Child Development and Care (CDC)
         - [NarrativeText] The Child Development and Care (CDC) program was e
         - [NarrativeText] The Michigan Department of Health and Human Servic
         - [NarrativeText] ACF develops and issues federal regulations that s
     - [Title] Program Goal
         - [NarrativeText] The Child Development and Care (CDC) program provi
         - [NarrativeText] The goal of the CDC program is to support low-inco
     - [Title] State Disability Assistance (SDA)
         - [NarrativeText] The State Disability Assistance (SDA) program was 
     - [Title] Program Goal
         - [NarrativeText] State Disability Assistance (SDA) provides financi
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 6 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [Title] POLICY MANUALS
     - [Title] All Programs
         - [NarrativeText] The Bridges Eligibility Manual (BEM), Bridges Admi
     - [Title] Bridges Eligibility Manual (BEM)
         - [NarrativeText] BEM contains policies and procedures related to de
     - [Title] Bridges Administrative Manual (BAM)
         - [NarrativeText] BAM contains policies and procedures related to ac
     - [Title] Bridges Glossary (BPG)
         - [NarrativeText] BPG contains definitions that assist with understa
     - [Title] Reference Manuals
         - [NarrativeText] The Reference Manuals include the following:
         - [ListItem] RFS Schedules. • RFT Tables.
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 7 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [Title] REVISIONS
         - [NarrativeText] The upper right-hand corner of each page lists rev
     - [Title] Bulletins
         - [NarrativeText] Bulletins contain a summary of all policy changes 
         - [ListItem] A list of the updated items. • A brief summary of 
         - [NarrativeText] Manual maintenance instructions automatically list
     - [Title] Change Bars
         - [NarrativeText] Revised manual pages contain vertical lines in the
     - [Title] Special Policy Bulletins
         - [NarrativeText] A special policy bulletin does not summarize an it
     - [Title] Public Access to Manuals
         - [NarrativeText] The MDHHS policy manuals are available to the publ
     - [Title] POLICY EXCEPTIONS
         - [NarrativeText] MDHHS and MDE policy is primarily the policy conta
         - [NarrativeText] Policy exceptions may be issued in case specific s
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 8 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [NarrativeText] eligibility results in the automated eligibility s
     - [ListItem] 
     - [NarrativeText] The eligibility results are incorrect or are incon
     - [ListItem] Eligibility must be manually determined and applie
     - [ListItem] Manual adjustments to federal or state FIP time cl
     - [ListItem] 
     - [NarrativeText] The department has been overturned in a hearing or
     - [NarrativeText] Policy exception decisions for case specific probl
     - [ListItem] 
     - [UncategorizedText] The Medical Services Administration within the Mic
     - [ListItem] 
     - [UncategorizedText] The Department of Education, Office of Great Start
     - [ListItem] 
     - [UncategorizedText] The Economic Stability Administration, in the MDHH
     - [NarrativeText] Policy issued on the DHS-1785 is official policy, 
     - [NarrativeText] There are three situations for which policy except
     - [ListItem] 
     - [NarrativeText] There is no existing policy in manuals or numbered
     - [ListItem] A policy exception is needed for use in a specific
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 9 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [ListItem] 
     - [NarrativeText] FIP, SDA, RCA, CDC, MA, and RMA Only A policy exce
     - [NarrativeText] Note: For some programs, central office must deter
     - [Title] Policy Exceptions Overrides
         - [NarrativeText] There are three situations in Bridges for which po
         - [ListItem] 
         - [NarrativeText] The automated determination in Bridges has produce
         - [ListItem] Policy must be published and made effective before
         - [ListItem] 
         - [NarrativeText] The published policy requires the use of an except
         - [NarrativeText] Note: For some programs, central office must also 
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 10 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [Title] EXCEPTION REQUESTS
     - [Title] Exceptions Not Covered by Published Policy
     - [Title] FIP, SDA, RCA/RMA, CDC and FAP only
         - [NarrativeText] Requests for a policy exception for a situation no
     - [Title] CDC
         - [UncategorizedText] Department of Education Office of Great Start Chil
     - [Title] Email: Policy-CDC@michigan.gov
     - [Title] FIP, SDA, and FAP
         - [UncategorizedText] Department of Health and Human Services Economic S
         - [UncategorizedText] Email: Policy-FIP-SDA@michigan.gov,
     - [Title] Policy-FAP@michigan.gov.
     - [Title] RCA/RMA
         - [UncategorizedText] Department of Labor and Economic Opportunity Offic
     - [Title] Email: LEO-RefugeeServices@michigan.gov
         - [NarrativeText] For the complete list of program policy email boxe
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 11 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [NarrativeText] Policy exception requests must include:
     - [ListItem] Case name (group member needing exception).
     - [ListItem] Case number.
     - [ListItem] Name and phone number of local office contact pers
     - [ListItem] A detailed description as to why the exception is 
     - [ListItem] What steps the local office has taken to resolve t
     - [ListItem] How the case fits into one of the three allowable 
     - [ListItem] Copies of all related material.
     - [Title] Medicaid only
         - [NarrativeText] Medicaid policy exception requests may be sent ID 
         - [UncategorizedText] Department of Health and Human Services Bureau of 
         - [NarrativeText] Medicaid exception requests may be faxed to: 517-2
         - [NarrativeText] Medicaid exceptions may be requested at eligibilit
         - [NarrativeText] Policy exception requests must include:
         - [ListItem] Beneficiary name.
         - [ListItem] Case number and beneficiary ID number.
         - [ListItem] Name and phone number of local office contact pers
         - [ListItem] A detailed description as to why the exception is 
         - [ListItem] What steps the local office has taken to resolve t
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 12 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [ListItem] Explanation of how the case fits into one of the t
     - [ListItem] Copies of all pertinent information.
     - [NarrativeText] Exceptions to provider or service policy, or prior
     - [NarrativeText] Medicaid policy exceptions are an internal process
     - [NarrativeText] Exception requests are not accepted from beneficia
     - [NarrativeText] Policy exceptions do not determine eligibility. An
     - [Title] All Programs except Medicaid
         - [NarrativeText] Policy exceptions override requests are generated 
         - [ListItem] A request is called in through administrative chan
         - [ListItem] A request is called in through administrative chan
         - [ListItem] • • • • • • A detailed description of the issue. •
         - [ListItem] A request is sent to the appropriate program polic
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 13 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [ListItem] 
     - [Title] The case number.
         - [ListItem] 
     - [Title] The Head of Household’s name.
         - [ListItem] 
     - [Title] The name and number of a local office contact pers
         - [ListItem] A detailed description regarding why the exception
         - [ListItem] What steps the local office has taken to handle th
         - [ListItem] Copies of all related materials.
         - [NarrativeText] If a request for a policy exceptions override is d
         - [NarrativeText] Policy exceptions for Medicaid are not processed i
     - [Title] Policy Exception Decisions
     - [Title] FIP, SDA, RCA/RMA, CDC, MA, and FAP only
         - [NarrativeText] When a policy exception is requested by a local of
         - [NarrativeText] MDHHS/MSA will issue policy exceptions via a fax o
         - [NarrativeText] Each DHS-1785 or MSA notice will be issued for a s
         - [NarrativeText] The DHS-1785 or MSA notice will be sent to the app
     - [Title] State Emergency Relief (SER) Only
     - [Title] See ERM 104, Exceptions to Official SER Policy.
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 14 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [Title] POLICY INTERPRETATIONS
     - [Title] All Programs
         - [NarrativeText] Implementation of existing policies in manuals, bu
         - [ListItem] Up to three designated staff persons from each loc
         - [ListItem] Program managers.
         - [ListItem] Partnership. Accountability. Training. Hope. (PATH
         - [ListItem] 
     - [Title] Food assistance management evaluators, (FAME).
         - [ListItem] AP specialists.
         - [NarrativeText] Program specific policy email box addresses are as
         - [ListItem] Policy-CDC@michigan.gov - Child Development and Ca
         - [ListItem] Policy-Employment@michigan.gov - FIP Employment an
         - [ListItem] MDHHS-Policy-FAPemployment@michigan.gov - FAP Empl
         - [ListItem] Policy-FAP@michigan.gov - Food Assistance Program 
         - [ListItem] Policy-FIP-SDA@michigan.gov - FIP and SDA Assistan
         - [ListItem] 
     - [Title] LEO-RefugeeServices@michigan.gov - RCA, RCA Employ
         - [ListItem] EligibilityPolicy@michigan.gov (Medicaid related q
         - [ListItem] Policy-SER@michigan.gov - State Emergency Relief P
         - [ListItem] MDHHS-EBT-Policy@michigan.gov - EBT Policy.
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 15 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [ListItem] MDHHS-MA-FAP-Trusts_Annuities@michigan.gov - Medic
     - [ListItem] MDHHS-Medicaid-Hearing-Reconsideration-
     - [NarrativeText] Requests@michigan.gov - Medicaid hearing reconside
     - [ListItem] MDHHS-Policy-Recoupment@michigan.gov - Recoupment 
     - [ListItem] Policy-Time-Limits@michigan.gov - TANF Out of Stat
     - [ListItem] MDHHS-Provider-Management@michigan.gov - Provider 
     - [ListItem] MDHHSVotes@michigan.gov.
     - [NarrativeText] Persons requesting policy clarifications are asked
     - [ListItem] • Programs needing clarification. • Manual item ne
     - [Title] Their name, telephone number and job title.
     - [Title] ADMINISTRATIVE HEARING DECISIONS
     - [Title] All Programs
         - [NarrativeText] Rulings and orders in a hearing decision are appli
     - [Title] QUALITY ASSURANCE ERRORS
     - [Title] All Programs
         - [NarrativeText] The Office of Quality Assurance and Internal Contr
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 16 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [NarrativeText] If OQAIC determines that an official policy is in 
     - [NarrativeText] BAM 320, Department Audits, contains information a
     - [Title] SUSPECTED CHILD ABUSE/NEGLECT
     - [Title] All Programs
         - [NarrativeText] MDHHS employees must report suspected child abuse 
         - [ListItem] 1. By phone to the MDHHS Centralized Intake Unit a
         - [ListItem] 2. Through MI Bridges for Mandatory Reporters. Mor
         - [NarrativeText] If making an oral report by telephone, within 72 h
         - [NarrativeText] The written report made online should contain the 
         - [UncategorizedText] For more information on mandatory reporting of chi
     - [Title] SUSPECTED ADULT ABUSE/NEGLECT
         - [NarrativeText] All MDHHS employees must report suspected adult ab
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 17 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [NarrativeText] See Administrative Policy Human Resources (APR) 20
     - [Title] LEGAL BASE
     - [Title] FIP
         - [NarrativeText] Social Security Act, Title IV, Part A, as amended 
     - [Title] SDA
     - [Title] Annual Appropriations Act Mich Admin Code, R 400.3
     - [Title] RCA/RMA
         - [UncategorizedText] 45 CFR 400.45 - 400.69 and 400.90 - 400.107
     - [Title] CDC
         - [NarrativeText] The Child Care and Development Block Grant (CCDBG)
     - [Title] MA
     - [Title] Social Security Act, Sections 1902 and 1905 42 CFR
         - [NarrativeText] FAP 7 CFR 271.1.3(a) 7 CFR 272.1(d) Food and Nutri
     - [Title] All Programs
         - [UncategorizedText] MCL 722.623(1), (2)
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES
 - [Header] BPB 2023-006
 - [Header] BEM 100
 - [Header] 18 of 18
 - [Header] INTRODUCTION
     - [UncategorizedText] 4-1-2023
     - [Title] PSF
         - [NarrativeText] Adoption Assistance and Child Welfare Act of 1980 
 - [Footer] BRIDGES ELIGIBILITY MANUAL
 - [Footer] STATE OF MICHIGAN
 - [Footer] DEPARTMENT OF HEALTH & HUMAN SERVICES


In [5]:
import re

# Example: iterating through text while ignoring elements part of the header or footer
flat_json = []
for element in pdf_json:
    if element['type'] == "Header" or element['type'] == "Footer":
        continue

    # Unstructured fails to categorize the date strings in the header,
    # so manually check for that and ignore those too
    if element['type'] == "UncategorizedText" and re.match(r'^\d{1,2}-\d{1,2}-\d{4}$', element['text']):
        continue

    element['children'] = []
    flat_json.append(element)

display_tree(flat_json)

 - [Title] OVERVIEW
 - [Title] Family Independence Program (FIP)
 - [NarrativeText] Temporary Assistance to Needy Families (TANF), cal
 - [NarrativeText] Each state must submit a state plan for FIP. State
 - [Title] Program Goal
 - [NarrativeText] The Family Independence Program (FIP) provides fin
 - [NarrativeText] Several nonfinancial and financial eligibility fac
 - [Title] Medical Assistance Program
 - [NarrativeText] The Medical Assistance Program was established by 
 - [NarrativeText] HHS develops and issues federal regulations that s
 - [Title] Program Goal
 - [NarrativeText] MA provides medical assistance to individuals and 
 - [NarrativeText] Disability related Medicaid and Group 2 Medicaid e
 - [Title] Food Assistance Program (FAP)
 - [NarrativeText] The Food Assistance Program (FAP) was established 
 - [NarrativeText] level. Within USDA, the Food and Nutrition Service
 - [Title] Program Goal
 - [NarrativeText] The purpose of the Food Assistance Program (FAP) i
 - [NarrativeText] FAP benefits are not considered income or assets f
 - [Title] Authorized Purchases
 - [NarrativeText] Food assistance benefits can be used to buy eligib
 - [NarrativeText] Eligible food includes:
 - [ListItem] Any food or food product intended for human consum
 - [ListItem] Seeds and plants to grow food for personal consump
 - [ListItem] Meals prepared by organizations approved by FNS as
 - [ListItem] Meals prepared and served to eligible residents by
 - [NarrativeText] Retail food stores include:
 - [ListItem] Recognized grocery stores.
 - [ListItem] House-to-house grocery vendors, such as milk and m
 - [ListItem] Nonprofit food purchasing ventures - private nonpr
 - [NarrativeText] Approved meal providers may include:
 - [ListItem] Communal dining facilities for elderly and disable
 - [ListItem] Restaurants which provide low-cost meals.
 - [ListItem] Meal delivery services - public or private nonprof
 - [Title] REFUGEE ASSISTANCE PROGRAMS
 - [NarrativeText] The refugee assistance programs were established b
 - [Title] Program Goal
 - [NarrativeText] The refugee assistance programs provide financial 
 - [NarrativeText] The Immigration and Nationality Act, the Code of F
 - [Title] Child Development and Care (CDC)
 - [NarrativeText] The Child Development and Care (CDC) program was e
 - [NarrativeText] The Michigan Department of Health and Human Servic
 - [NarrativeText] ACF develops and issues federal regulations that s
 - [Title] Program Goal
 - [NarrativeText] The Child Development and Care (CDC) program provi
 - [NarrativeText] The goal of the CDC program is to support low-inco
 - [Title] State Disability Assistance (SDA)
 - [NarrativeText] The State Disability Assistance (SDA) program was 
 - [Title] Program Goal
 - [NarrativeText] State Disability Assistance (SDA) provides financi
 - [Title] POLICY MANUALS
 - [Title] All Programs
 - [NarrativeText] The Bridges Eligibility Manual (BEM), Bridges Admi
 - [Title] Bridges Eligibility Manual (BEM)
 - [NarrativeText] BEM contains policies and procedures related to de
 - [Title] Bridges Administrative Manual (BAM)
 - [NarrativeText] BAM contains policies and procedures related to ac
 - [Title] Bridges Glossary (BPG)
 - [NarrativeText] BPG contains definitions that assist with understa
 - [Title] Reference Manuals
 - [NarrativeText] The Reference Manuals include the following:
 - [ListItem] RFS Schedules. • RFT Tables.
 - [Title] REVISIONS
 - [NarrativeText] The upper right-hand corner of each page lists rev
 - [Title] Bulletins
 - [NarrativeText] Bulletins contain a summary of all policy changes 
 - [ListItem] A list of the updated items. • A brief summary of 
 - [NarrativeText] Manual maintenance instructions automatically list
 - [Title] Change Bars
 - [NarrativeText] Revised manual pages contain vertical lines in the
 - [Title] Special Policy Bulletins
 - [NarrativeText] A special policy bulletin does not summarize an it
 - [Title] Public Access to Manuals
 - [NarrativeText] The MDHHS policy manuals are available to the publ
 - [Title] POLICY EXCEPTIONS
 - [NarrativeText] MDHHS and MDE policy is primarily the policy conta
 - [NarrativeText] Policy exceptions may be issued in case specific s
 - [NarrativeText] eligibility results in the automated eligibility s
 - [ListItem] 
 - [NarrativeText] The eligibility results are incorrect or are incon
 - [ListItem] Eligibility must be manually determined and applie
 - [ListItem] Manual adjustments to federal or state FIP time cl
 - [ListItem] 
 - [NarrativeText] The department has been overturned in a hearing or
 - [NarrativeText] Policy exception decisions for case specific probl
 - [ListItem] 
 - [UncategorizedText] The Medical Services Administration within the Mic
 - [ListItem] 
 - [UncategorizedText] The Department of Education, Office of Great Start
 - [ListItem] 
 - [UncategorizedText] The Economic Stability Administration, in the MDHH
 - [NarrativeText] Policy issued on the DHS-1785 is official policy, 
 - [NarrativeText] There are three situations for which policy except
 - [ListItem] 
 - [NarrativeText] There is no existing policy in manuals or numbered
 - [ListItem] A policy exception is needed for use in a specific
 - [ListItem] 
 - [NarrativeText] FIP, SDA, RCA, CDC, MA, and RMA Only A policy exce
 - [NarrativeText] Note: For some programs, central office must deter
 - [Title] Policy Exceptions Overrides
 - [NarrativeText] There are three situations in Bridges for which po
 - [ListItem] 
 - [NarrativeText] The automated determination in Bridges has produce
 - [ListItem] Policy must be published and made effective before
 - [ListItem] 
 - [NarrativeText] The published policy requires the use of an except
 - [NarrativeText] Note: For some programs, central office must also 
 - [Title] EXCEPTION REQUESTS
 - [Title] Exceptions Not Covered by Published Policy
 - [Title] FIP, SDA, RCA/RMA, CDC and FAP only
 - [NarrativeText] Requests for a policy exception for a situation no
 - [Title] CDC
 - [UncategorizedText] Department of Education Office of Great Start Chil
 - [Title] Email: Policy-CDC@michigan.gov
 - [Title] FIP, SDA, and FAP
 - [UncategorizedText] Department of Health and Human Services Economic S
 - [UncategorizedText] Email: Policy-FIP-SDA@michigan.gov,
 - [Title] Policy-FAP@michigan.gov.
 - [Title] RCA/RMA
 - [UncategorizedText] Department of Labor and Economic Opportunity Offic
 - [Title] Email: LEO-RefugeeServices@michigan.gov
 - [NarrativeText] For the complete list of program policy email boxe
 - [NarrativeText] Policy exception requests must include:
 - [ListItem] Case name (group member needing exception).
 - [ListItem] Case number.
 - [ListItem] Name and phone number of local office contact pers
 - [ListItem] A detailed description as to why the exception is 
 - [ListItem] What steps the local office has taken to resolve t
 - [ListItem] How the case fits into one of the three allowable 
 - [ListItem] Copies of all related material.
 - [Title] Medicaid only
 - [NarrativeText] Medicaid policy exception requests may be sent ID 
 - [UncategorizedText] Department of Health and Human Services Bureau of 
 - [NarrativeText] Medicaid exception requests may be faxed to: 517-2
 - [NarrativeText] Medicaid exceptions may be requested at eligibilit
 - [NarrativeText] Policy exception requests must include:
 - [ListItem] Beneficiary name.
 - [ListItem] Case number and beneficiary ID number.
 - [ListItem] Name and phone number of local office contact pers
 - [ListItem] A detailed description as to why the exception is 
 - [ListItem] What steps the local office has taken to resolve t
 - [ListItem] Explanation of how the case fits into one of the t
 - [ListItem] Copies of all pertinent information.
 - [NarrativeText] Exceptions to provider or service policy, or prior
 - [NarrativeText] Medicaid policy exceptions are an internal process
 - [NarrativeText] Exception requests are not accepted from beneficia
 - [NarrativeText] Policy exceptions do not determine eligibility. An
 - [Title] All Programs except Medicaid
 - [NarrativeText] Policy exceptions override requests are generated 
 - [ListItem] A request is called in through administrative chan
 - [ListItem] A request is called in through administrative chan
 - [ListItem] • • • • • • A detailed description of the issue. •
 - [ListItem] A request is sent to the appropriate program polic
 - [ListItem] 
 - [Title] The case number.
 - [ListItem] 
 - [Title] The Head of Household’s name.
 - [ListItem] 
 - [Title] The name and number of a local office contact pers
 - [ListItem] A detailed description regarding why the exception
 - [ListItem] What steps the local office has taken to handle th
 - [ListItem] Copies of all related materials.
 - [NarrativeText] If a request for a policy exceptions override is d
 - [NarrativeText] Policy exceptions for Medicaid are not processed i
 - [Title] Policy Exception Decisions
 - [Title] FIP, SDA, RCA/RMA, CDC, MA, and FAP only
 - [NarrativeText] When a policy exception is requested by a local of
 - [NarrativeText] MDHHS/MSA will issue policy exceptions via a fax o
 - [NarrativeText] Each DHS-1785 or MSA notice will be issued for a s
 - [NarrativeText] The DHS-1785 or MSA notice will be sent to the app
 - [Title] State Emergency Relief (SER) Only
 - [Title] See ERM 104, Exceptions to Official SER Policy.
 - [Title] POLICY INTERPRETATIONS
 - [Title] All Programs
 - [NarrativeText] Implementation of existing policies in manuals, bu
 - [ListItem] Up to three designated staff persons from each loc
 - [ListItem] Program managers.
 - [ListItem] Partnership. Accountability. Training. Hope. (PATH
 - [ListItem] 
 - [Title] Food assistance management evaluators, (FAME).
 - [ListItem] AP specialists.
 - [NarrativeText] Program specific policy email box addresses are as
 - [ListItem] Policy-CDC@michigan.gov - Child Development and Ca
 - [ListItem] Policy-Employment@michigan.gov - FIP Employment an
 - [ListItem] MDHHS-Policy-FAPemployment@michigan.gov - FAP Empl
 - [ListItem] Policy-FAP@michigan.gov - Food Assistance Program 
 - [ListItem] Policy-FIP-SDA@michigan.gov - FIP and SDA Assistan
 - [ListItem] 
 - [Title] LEO-RefugeeServices@michigan.gov - RCA, RCA Employ
 - [ListItem] EligibilityPolicy@michigan.gov (Medicaid related q
 - [ListItem] Policy-SER@michigan.gov - State Emergency Relief P
 - [ListItem] MDHHS-EBT-Policy@michigan.gov - EBT Policy.
 - [ListItem] MDHHS-MA-FAP-Trusts_Annuities@michigan.gov - Medic
 - [ListItem] MDHHS-Medicaid-Hearing-Reconsideration-
 - [NarrativeText] Requests@michigan.gov - Medicaid hearing reconside
 - [ListItem] MDHHS-Policy-Recoupment@michigan.gov - Recoupment 
 - [ListItem] Policy-Time-Limits@michigan.gov - TANF Out of Stat
 - [ListItem] MDHHS-Provider-Management@michigan.gov - Provider 
 - [ListItem] MDHHSVotes@michigan.gov.
 - [NarrativeText] Persons requesting policy clarifications are asked
 - [ListItem] • Programs needing clarification. • Manual item ne
 - [Title] Their name, telephone number and job title.
 - [Title] ADMINISTRATIVE HEARING DECISIONS
 - [Title] All Programs
 - [NarrativeText] Rulings and orders in a hearing decision are appli
 - [Title] QUALITY ASSURANCE ERRORS
 - [Title] All Programs
 - [NarrativeText] The Office of Quality Assurance and Internal Contr
 - [NarrativeText] If OQAIC determines that an official policy is in 
 - [NarrativeText] BAM 320, Department Audits, contains information a
 - [Title] SUSPECTED CHILD ABUSE/NEGLECT
 - [Title] All Programs
 - [NarrativeText] MDHHS employees must report suspected child abuse 
 - [ListItem] 1. By phone to the MDHHS Centralized Intake Unit a
 - [ListItem] 2. Through MI Bridges for Mandatory Reporters. Mor
 - [NarrativeText] If making an oral report by telephone, within 72 h
 - [NarrativeText] The written report made online should contain the 
 - [UncategorizedText] For more information on mandatory reporting of chi
 - [Title] SUSPECTED ADULT ABUSE/NEGLECT
 - [NarrativeText] All MDHHS employees must report suspected adult ab
 - [NarrativeText] See Administrative Policy Human Resources (APR) 20
 - [Title] LEGAL BASE
 - [Title] FIP
 - [NarrativeText] Social Security Act, Title IV, Part A, as amended 
 - [Title] SDA
 - [Title] Annual Appropriations Act Mich Admin Code, R 400.3
 - [Title] RCA/RMA
 - [UncategorizedText] 45 CFR 400.45 - 400.69 and 400.90 - 400.107
 - [Title] CDC
 - [NarrativeText] The Child Care and Development Block Grant (CCDBG)
 - [Title] MA
 - [Title] Social Security Act, Sections 1902 and 1905 42 CFR
 - [NarrativeText] FAP 7 CFR 271.1.3(a) 7 CFR 272.1(d) Food and Nutri
 - [Title] All Programs
 - [UncategorizedText] MCL 722.623(1), (2)
 - [Title] PSF
 - [NarrativeText] Adoption Assistance and Child Welfare Act of 1980 


In [10]:
import pprint
# Example element
for element in pdf_json:
    if 'MDHHS Policy Manuals;' in element['text']:
        pprint.pprint(element)
        break

{'children': [],
 'element_id': 'f14565c3d292bebb36a5c4ab78315e72',
 'metadata': {'coordinates': {'layout_height': 792,
                              'layout_width': 612,
                              'points': ((198.05, 532.03),
                                         (198.05, 585.46),
                                         (546.6860000000001, 585.46),
                                         (546.6860000000001, 532.03)),
                              'system': 'PixelSpace'},
              'file_directory': '/tmp',
              'filename': 'tmpjpuz0g4i.pdf',
              'filetype': 'application/pdf',
              'languages': ['eng'],
              'last_modified': '2024-08-21T16:04:57',
              'links': [{'start_index': 129,
                         'text': 'MDHHS Policy Manuals ;',
                         'url': 'http://www.michigan.gov/mdhhs'}],
              'page_number': 7,
              'parent_id': '47aa8214199fdffc458e0fdf6d918150'},
 'text': 'The MDHHS policy m